In [2]:
# Make sure you run the get_data.sh script before getting started!

In [28]:
# read the text file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [29]:
print(f"dataset length in characters: {len(text)}")

dataset length in characters: 1115394


In [30]:
# output the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [31]:
# list the unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [32]:
# Tokenization (naive character level tokenizer)
# other methods include SentencePiece, tiktoken

# create a dictionary to map character to integer (corresponding to the index in chars)
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

# create encoder/decoder lambda functions
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [33]:
# encode the entire dataset and display the first 100 tokens
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [34]:
# Train/Validation Split
n = int(0.9*len(data)) # number of tokens to be used for training (first 90%)
train_data = data[:n]
val_data = data[n:]

In [35]:
block_size = 8 # the size of the training data CHUNK to use for training
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [36]:
# Example of using different amounts of context to predict target using the same data CHUNK
# NOTE: is this essentially masking?
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [37]:
# batch dimension: used to parallelize GPU training processing (stack up multiple chunks to be processed at the same time)

torch.manual_seed(1337) # set seed for reproducibility with Karpathy's video
batch_size = 4 # how many independent sequences will be processed in parallel (batch_dimension)
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
    # generate a random, small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data # switch dataset whether we want training or validation data
    ix = torch.randint(len(data)-block_size, (batch_size,)) # get batch_size random 'cuts' in the dataset (starting indices/offsets for our chunks basically)
    x = torch.stack([data[i:i+block_size] for i in ix]) # generate the chunks based on the 'cuts' and the block_size -> do it for each of the batches and stack them together in one tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [38]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

# essentially a table of the likelihood of token B given token A, for the entire set of tokens in the vocabulary (predict the next token given the current token)
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and trargets are both (B,T) tensor of integers
        # B: # of batches (4), T: # of tokens per batch (8)
        logits = self.token_embedding_table(idx) # (B,T,C) -> C: channel (vocab size)

        if targets is None:
            loss=None
        else:
            # re-shape logits and targets to be (B*T, C) and (B*T, 1) respectively
            B , T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)    
            # measure loss using negative log-likelihood
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) 
            # focus only on the last time step (since we just want to GENERATE the NEXT token, not validating or anything)
            logits = logits[:, -1, :] # become (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution (next token for each batch)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [41]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


# The Mathematical trick in Self-Attention

In [44]:
import torch
torch.manual_seed(1337)
B,T,C = 4,8,2 # Batch, Time, Channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [45]:
# We want x[b,t] = mean_{i<=t} x[b,i]
# NOTE: xbow -> x "bag of words"

# VERSION 1: Naiive with loops
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

# For loops aren't efficient, how do we "matrixify" this? (leveraging linear algebra)
print(x[0])
print(xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [46]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3)) # lower diagonal matrix is the trick!, but this makes it just a sum
a = a / torch.sum(a, 1, keepdim=True) # to make it a WEIGHTED sum, normalize the rows so they add to 1 (in essence, the average)
b = torch.randint(0,10,(3,2)).float() 
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


## Applying the trick

In [47]:
# Version 2: Using lower triangular normalized matrix
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (*B*, T, T) @ (B, T, C) --> (B,T,C)

In [52]:
# Version 3: use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T)) # for simple bag of words, wei is all 0s. But for attention, eventually these weights will haev a value that associates its "affinity" more 
wei = wei.masked_fill(tril == 0, float('-inf')) # ensure future words have zero correlation/influence on aggregation
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x